In [1]:
import pandas as pd
import numpy as np
from glob import glob
from pathlib import Path
import pynmea2
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
from tqdm.notebook import tqdm
import copy


In [8]:
files_list = glob(r'../../Data/Caravela/CaravelaPCS/Datalog/Y20/*/*'+ '/recordDataPort_D*')  
files_list.sort()
li = []

for filename in tqdm(range(4,len(files_list))): # take files from Caravelas launch onwards
    with open(files_list[filename],'r') as myfile:
        content = myfile.readlines()[1:-1] #skip first line of file as this is likey to have not been written correctly
    frame = pd.DataFrame(data = np.array(content), columns= ['original'])
    li.append(frame)
df = pd.concat(li, axis=0,ignore_index=True)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfe in position 2166: invalid start byte

In [7]:
files_list

['../../Data/Caravela/CaravelaPCS/Datalog/Y20/M01/D15/recordDataPort_D_001.txt',
 '../../Data/Caravela/CaravelaPCS/Datalog/Y20/M01/D15/recordDataPort_D_002.txt',
 '../../Data/Caravela/CaravelaPCS/Datalog/Y20/M01/D16/recordDataPort_D_001.txt',
 '../../Data/Caravela/CaravelaPCS/Datalog/Y20/M01/D19/recordDataPort_D_001.txt',
 '../../Data/Caravela/CaravelaPCS/Datalog/Y20/M01/D22/recordDataPort_D_001.txt',
 '../../Data/Caravela/CaravelaPCS/Datalog/Y20/M01/D23/recordDataPort_D_001.txt',
 '../../Data/Caravela/CaravelaPCS/Datalog/Y20/M01/D24/recordDataPort_D_001.txt',
 '../../Data/Caravela/CaravelaPCS/Datalog/Y20/M01/D25/recordDataPort_D_001.txt',
 '../../Data/Caravela/CaravelaPCS/Datalog/Y20/M01/D26/recordDataPort_D_001.txt',
 '../../Data/Caravela/CaravelaPCS/Datalog/Y20/M01/D27/recordDataPort_D_001.txt',
 '../../Data/Caravela/CaravelaPCS/Datalog/Y20/M01/D28/recordDataPort_D_001.txt',
 '../../Data/Caravela/CaravelaPCS/Datalog/Y20/M01/D29/recordDataPort_D_001.txt',
 '../../Data/Caravela/Carave